# Concurrency with concurrent.futures

If you write a lot of concurrent code you'll notice that you end up having to do various types of tasks over and over again, including:

- Checking if a computation is finished
- Finding out the result of the computation
- Checking if a task is running
- Cancelling a task
- Dealing with timeout conditions

Because all these things are so common there was a proposal to make an interface in the Python standard library: https://www.python.org/dev/peps/pep-3148/

This interface is found in `concurrent.futures` since Python 3.2.


## Backported version
If you need to get a backported version for an older version you will have to first install the backported version with:

In [ ]:
pip install futures

Note however that if you are running Python 2.7 this will not support `ProcessPoolExecutor` because of the following open issue https://github.com/agronholm/pythonfutures/issues/29 and https://bugs.python.org/issue9205 (If you need this please consider porting your code to a more recent Python 3 version)

If you are attempting to support both Python 2 and 3 from the same code base you will have to make sure that `futures` is only installed on Python 2.7, you can do this in your setup.py as follows:

In [ ]:
setup(
    ...
    install_requires={
        'futures; python_version == "2.7"'
    }
)

## Scheduling a task to run concurrently

When you deal with any form of concurrency you have to not only specify what code will run, you also have to specify when and how that code will run.

`concurrent.futures` gives us an abstract `Executor` interface which will allow us to schedule tasks to run. Concrete implementations of this interface will specify how these tasks will be executed. We will use a `ThreadPoolExecutor` for many of these examples.

In [21]:
import concurrent.futures
import threading
import time

def task(time_to_wait: int):
    """A task that will take `time_to_wait` seconds to complete"""
    print(f"Currently running on thread {threading.current_thread().name}")
    time.sleep(time_to_wait)
    print(f"Task completed on thread {threading.current_thread().name}")
    return time_to_wait

exe = concurrent.futures.ThreadPoolExecutor(max_workers=2)
print("Main thread")
fut = exe.submit(task, 3)
print(type(fut))
print(fut.done())
time.sleep(5)
print(fut.done())
print("Result:", fut.result())
exe.shutdown(wait=True) # good to clean up properly

Main thread
Currently running on thread ThreadPoolExecutor-5_0
<class 'concurrent.futures._base.Future'>
False
Task completed on thread ThreadPoolExecutor-5_0
True
Result: 3


## The Future class
When we submit a task to run in an Executor we get a `Future` result returned from the executor. This allows us to deal with the asynchronous nature of the task.

There's a few parts involved in using such an object.

Since a `Future` object is running asynchronously we will need to check if the task is completed before we can use the results.

- `fut.done()` will return if the task has completed or not.
- `fut.cancel()` will attempt to cancel the task, this can only be done before the execution of the task has commenced.
- `fut.cancelled()` will return if the task was cancelled.
- `fut.result()` will wait indefinitely for the task to complete, NOTE this will block

If we want to wait a shorter amount of time for the tasks to complete then we should specify the `timeout` argument for the `result()` method call so we don't wait forever.

## A motivating example: a group of slow tasks
Some tasks take a lot of time to execute but don't necessarily involve a lot of resources. One good example is Network latency bound tasks another is waiting on a remote end to perform a task.

A good example of this is looking up a slow API endpoint, in many situations you don't want to have your codes execution be blocked while waiting on a slow network API call. A good example of an API endpoint that takes a while to return data is from the HTTP requests testing server (httpbin). In particular we will make use of the endpoint `/delay` that's only purpose is to delay for the given number of seconds on the server before it returns us the data from the request.

In [7]:
import requests
slow_url_1 = "https://httpbin.org/delay/1"
slow_url_5 = "https://httpbin.org/delay/5"

In [8]:
response = requests.get(slow_url_1)
response.elapsed

datetime.timedelta(seconds=2, microseconds=88922)

In [9]:
response = requests.get(slow_url_5)
response.elapsed

datetime.timedelta(seconds=6, microseconds=92957)

In [11]:
response.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.19.1'},
 'origin': '123.243.137.65, 123.243.137.65',
 'url': 'https://httpbin.org/delay/5'}

## API calls: The slow synchronous way
We could call these endpoints then store the results from these calls synchronously, but we will end up having to wait a while:

In [35]:
%%timeit
import requests
URLS = [
    "https://httpbin.org/delay/1",
    "https://httpbin.org/delay/3",
    "https://httpbin.org/delay/5",
]

def get_results(url):
    """Fetch results from URL via a GET request"""
    response = requests.get(URL)
    return response.json()

results = {}
for URL in URLS:
    results[URL] = get_results(URL)

12.3 s ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
results

{'https://httpbin.org/delay/1': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/1'},
 'https://httpbin.org/delay/3': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/3'},
 'https://httpbin.org/delay/5': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/5'}}

## Managing the flow of control: callbacks

As you can see with those requests it takes a long time for us to get any results due to the combination of network latency and the time we have to wait while the server is "processing" our request. If you have a large number of requests you'll end up potentially waiting a *long* wall-clock-time. If we are to run these tasks in parallel then we could see a lot of speed up since calling the API does not rely on any other pervious call ocurring in any order. This does introduce an additional complexity, because any API call can complete in any order we need some way of compiling the results, the simple loop from before will not be able to do this.

Perhaps the simplest way to deal with this is to use a callback function, the nature of such a function is that when the task is done it "calls back" to the function we provided. In that callback function we can then process the results as we wish. `fut.add_done_callback` will allow us to register an existing function as a callback function on a task completion.

When we specify a function as a callback it receives the future that called it as a parameter. If we need to do something when the task is completed, such as store results, that requires the parameters used when calling the future in the first place we will have to keep track of them. Futures can be stored as dictionary keys so using them in a dictionary as the key with the information that is needed to be stored for later is a good approach. That's the purpose of `jobs` in the following example:

In [37]:
%%timeit
import requests
URLS = [
    "https://httpbin.org/delay/1",
    "https://httpbin.org/delay/3",
    "https://httpbin.org/delay/5",
]

results = {}

def get_results(url):
    """Fetch results from URL via a GET request"""
    response = requests.get(URL)
    return response.json()
    
def url_done(fut):
    """Callback when the future is done"""
    url = jobs[fut]
    results[url] = fut.result()
    
exe = concurrent.futures.ThreadPoolExecutor(max_workers=5)

jobs = {}
for URL in URLS:
    fut = exe.submit(get_results, URL)
    jobs[fut] = URL
    fut.add_done_callback(url_done)
exe.shutdown(wait=True) # good to clean up properly

6.11 s ± 8.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
results

{'https://httpbin.org/delay/1': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/1'},
 'https://httpbin.org/delay/3': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/3'},
 'https://httpbin.org/delay/5': {'args': {},
  'data': '',
  'files': {},
  'form': {},
  'headers': {'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Host': 'httpbin.org',
   'User-Agent': 'python-requests/2.19.1'},
  'origin': '123.243.137.65, 123.243.137.65',
  'url': 'https://httpbin.org/delay/5'}}

## Threadpool context manager

One thing you may find useful is to use the Context manager for the Executor, within the block it defines you will be able to submit jobs to it.

This context manager will call `executor.shutdown(wait=True)` when the block leaves.

In [23]:
import time
import threading

def slow_square(number, delay):
    """Simulate a slow calculation"""
    print(f"Calculating the square of {number} after {delay} seconds",
          f"on thread {threading.current_thread().name}")
    time.sleep(delay)
    return number ** 2

In [24]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=5) as executor:
    s1 = executor.submit(slow_square, 1, 2)
    s2 = executor.submit(slow_square, 3, 2)
    s3 = executor.submit(slow_square, 4, 8)

print("Results:", s1.result(), s2.result(), s3.result())

Calculating the square of 1 after 2 seconds on thread ThreadPoolExecutor-6_0
Calculating the square of 3 after 2 seconds on thread ThreadPoolExecutor-6_1
Calculating the square of 4 after 8 seconds on thread ThreadPoolExecutor-6_2
Results: 1 9 16


## Map

`executor.map` will calculate multiple results much like `map` but across multiple threads.

Note that there's one annoyance when you have to pass multiple arguments to a function, due to the limited API this requires a small workaround, since for `map` to be able to differentiate between the following set of calls:

1. `arg1 = [(1, 2), (3, 2), (4, 8)]`
2. `arg1 = (1, 2); arg2 = (3, 2); arg3 = (4, 8)`

We use `*zip(*args)` to specify that we wanted the second option with 3 function calls.

In [41]:
import time
from concurrent.futures import ThreadPoolExecutor
args = [(1, 2), (3, 2), (4, 8)]

with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(slow_square, args) # BUG!

for result in results:
    print(result)

TypeError: slow_square() missing 1 required positional argument: 'delay'

In [25]:
import time
from concurrent.futures import ThreadPoolExecutor
args = [(1, 2), (3, 2), (4, 8)]

with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(slow_square, *zip(*args))

for result in results:
    print(result)

1
9
16


## Exercise: weather exercise with a thread pool
Earlier in this course we showed you how you could look up the current weather in a variety of cities, this is something that has a very substantial amount of network latency.

Since this is a highly IO bound task we have a good opportunity to use concurrency to speed things up (see Concurrency overview to see other options).

Note: you may use a rate-limited API, if this is the case you will want to make sure you don't make too many calls at once. OpenWeatherMap is limited to 60 API calls per minute on the free plan so make sure you don't have too long a list of cities when running concurrently.

- Use a `ThreadPoolExecutor` to send queries to the OpenWeatherMap API in parallel
- Extension: Handle timeout error cases
- Extension: Gracefully handle cancelling the remaining tasks if there's a `KeyboardInterrupt`

In [ ]:
# see solutions/threadpool_weather.py